# Multinomial Logistic Regression

El ejercicio: 

***Los estudiantes eligen diferentes tipos de programa (prog). Dicha elección puede ser modelada según diferentes scores y variables.***

In [1]:
#install.packages('foreign') #lee diversos formatos de diferentes programas estadístico
library(foreign) 
library(dplyr) 

Warning message:
"package 'dplyr' was built under R version 4.0.5"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [3]:
df=read.dta("https://stats.idre.ucla.edu/stat/data/hsbdemo.dta")
tail(df,10)

,id,female,ses,schtyp,prog,read,write,math,science,socst,honors,awards,cid
,<dbl>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<int>
191,180,female,high,private,academic,71,65,69,58,71,enrolled,5,20
192,82,female,high,public,academic,68,62,65,69,61,enrolled,3,20
193,174,male,middle,private,academic,68,59,71,66,56,not enrolled,2,20
194,95,male,high,public,academic,73,60,71,61,71,enrolled,2,20
195,61,female,high,public,academic,76,63,60,67,66,enrolled,4,20
196,100,female,high,public,academic,63,65,71,69,71,enrolled,5,20
197,143,male,middle,public,vocation,63,63,75,72,66,enrolled,4,20
198,68,male,middle,public,academic,73,67,71,63,66,enrolled,7,20
199,57,female,middle,public,academic,71,65,72,66,56,enrolled,5,20


In [4]:
str(df) #es para ver el tipo de variable en mi base

'data.frame':	200 obs. of  13 variables:
 $ id     : num  45 108 15 67 153 51 164 133 2 53 ...
 $ female : Factor w/ 2 levels "male","female": 2 1 1 1 1 2 1 1 2 1 ...
 $ ses    : Factor w/ 3 levels "low","middle",..: 1 2 3 1 2 3 2 2 2 2 ...
 $ schtyp : Factor w/ 2 levels "public","private": 1 1 1 1 1 1 1 1 1 1 ...
 $ prog   : Factor w/ 3 levels "general","academic",..: 3 1 3 3 3 1 3 3 3 3 ...
 $ read   : num  34 34 39 37 39 42 31 50 39 34 ...
 $ write  : num  35 33 39 37 31 36 36 31 41 37 ...
 $ math   : num  41 41 44 42 40 42 46 40 33 46 ...
 $ science: num  29 36 26 33 39 31 39 34 42 39 ...
 $ socst  : num  26 36 42 32 51 39 46 31 41 31 ...
 $ honors : Factor w/ 2 levels "not enrolled",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ awards : num  0 0 0 0 0 0 0 0 0 0 ...
 $ cid    : int  1 1 1 1 1 1 1 1 1 1 ...
 - attr(*, "datalabel")= chr "highschool and beyond (200 cases)"
 - attr(*, "time.stamp")= chr "30 Oct 2009 14:13"
 - attr(*, "formats")= chr [1:13] "%9.0g" "%9.0g" "%9.0g" "%9.0g" ...
 - attr(

In [6]:
#Frecuencia de la variable respuesta
table(df$prog)


 general academic vocation 
      45      105       50 

In [9]:
df%>%
    group_by(prog,ses)%>%
    summarize(count_prog=n(),
             mean_read=round(mean(read),2),
             mean_write=round(mean(write),2),
             mean_math=round(mean(math),2))

`summarise()` has grouped output by 'prog'. You can override using the `.groups` argument.



prog,ses,count_prog,mean_read,mean_write,mean_math
<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>
general,low,16,47.56,51.44,47.56
general,middle,20,51.05,49.60,51.10
general,high,9,50.78,55.00,52.00
academic,low,19,53.00,53.63,53.95
academic,middle,44,54.68,56.36,56.32
academic,high,42,59.14,57.33,58.43
vocation,low,12,41.75,44.75,43.75
vocation,middle,31,47.52,47.13,47.10
vocation,high,7,48.00,48.57,48.00


**Tarea: Hacer gráficas descriptivas**

In [7]:
#install.packages('nnet')
library(nnet)

Warning message:
"package 'nnet' was built under R version 4.0.5"


Tenemos que revisar los valores que toma mi variable prog (niveles/levels)

In [8]:
levels(df$prog)

[1] "general"  "academic" "vocation"

Tenemos que definir nuestro valor pivote en la variable, tenemos dos rutas:

1. Hacerlo antes de crear nuestro modelo multinomial (función relevel).
2. Hacerlo durante la creación del modelo multinomial.

1. Antes del modelo multinomial

In [10]:
df$prog_2=relevel(df$prog,ref='academic')
head(df,10)

,id,female,ses,schtyp,prog,read,write,math,science,socst,honors,awards,cid,prog_2
,<dbl>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<int>,<fct>
1,45,female,low,public,vocation,34,35,41,29,26,not enrolled,0,1,vocation
2,108,male,middle,public,general,34,33,41,36,36,not enrolled,0,1,general
3,15,male,high,public,vocation,39,39,44,26,42,not enrolled,0,1,vocation
4,67,male,low,public,vocation,37,37,42,33,32,not enrolled,0,1,vocation
5,153,male,middle,public,vocation,39,31,40,39,51,not enrolled,0,1,vocation
6,51,female,high,public,general,42,36,42,31,39,not enrolled,0,1,general
7,164,male,middle,public,vocation,31,36,46,39,46,not enrolled,0,1,vocation
8,133,male,middle,public,vocation,50,31,40,34,31,not enrolled,0,1,vocation
9,2,female,middle,public,vocation,39,41,33,42,41,not enrolled,0,1,vocation


In [11]:
str(df)

'data.frame':	200 obs. of  14 variables:
 $ id     : num  45 108 15 67 153 51 164 133 2 53 ...
 $ female : Factor w/ 2 levels "male","female": 2 1 1 1 1 2 1 1 2 1 ...
 $ ses    : Factor w/ 3 levels "low","middle",..: 1 2 3 1 2 3 2 2 2 2 ...
 $ schtyp : Factor w/ 2 levels "public","private": 1 1 1 1 1 1 1 1 1 1 ...
 $ prog   : Factor w/ 3 levels "general","academic",..: 3 1 3 3 3 1 3 3 3 3 ...
 $ read   : num  34 34 39 37 39 42 31 50 39 34 ...
 $ write  : num  35 33 39 37 31 36 36 31 41 37 ...
 $ math   : num  41 41 44 42 40 42 46 40 33 46 ...
 $ science: num  29 36 26 33 39 31 39 34 42 39 ...
 $ socst  : num  26 36 42 32 51 39 46 31 41 31 ...
 $ honors : Factor w/ 2 levels "not enrolled",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ awards : num  0 0 0 0 0 0 0 0 0 0 ...
 $ cid    : int  1 1 1 1 1 1 1 1 1 1 ...
 $ prog_2 : Factor w/ 3 levels "academic","general",..: 3 2 3 3 3 2 3 3 3 3 ...
 - attr(*, "datalabel")= chr "highschool and beyond (200 cases)"
 - attr(*, "time.stamp")= chr "30 Oct 2009 14:13

Crear mi primer modelo multinomial :)

In [15]:
multi1=multinom(prog_2~ses+write,data=df) #nuestra variable respuesta es la relevel
summary(multi1)

# weights:  15 (8 variable)
initial  value 219.722458 
iter  10 value 179.982880
final  value 179.981726 
converged


Call:
multinom(formula = prog_2 ~ ses + write, data = df)

Coefficients:
         (Intercept)  sesmiddle    seshigh      write
general     2.852198 -0.5332810 -1.1628226 -0.0579287
vocation    5.218260  0.2913859 -0.9826649 -0.1136037

Std. Errors:
         (Intercept) sesmiddle   seshigh      write
general     1.166441 0.4437323 0.5142196 0.02141097
vocation    1.163552 0.4763739 0.5955665 0.02221996

Residual Deviance: 359.9635 
AIC: 375.9635 

Para B=-0.53, sería algo así "la B asociada a un estudiante que elige bachillerato general, en comparación con académico y su nivel socioeconómico es medio (en comparación con una persona que proviene de un nivel socioeconómico bajo). 

In [13]:
exp(-0.5332810)

[1] 0.5866769

In [14]:
exp(0.2913859)

[1] 1.338281

Opción 2: Nivelar (definir) la etiqueta pivote durante la creación del modelo.

In [19]:
multi2=multinom(prog~ses+write,data=df)#,contrasts='vocation')
summary(multi2)

# weights:  15 (8 variable)
initial  value 219.722458 
iter  10 value 179.985215
final  value 179.981726 
converged


Call:
multinom(formula = prog ~ ses + write, data = df)

Coefficients:
         (Intercept) sesmiddle   seshigh       write
academic   -2.851973 0.5332914 1.1628257  0.05792480
vocation    2.366097 0.8246384 0.1802176 -0.05567514

Std. Errors:
         (Intercept) sesmiddle   seshigh      write
academic    1.166437 0.4437319 0.5142215 0.02141092
vocation    1.174251 0.4901237 0.6484508 0.02333135

Residual Deviance: 359.9635 
AIC: 375.9635 

In [42]:
estadisticos=summary(multi1)$coefficients/summary(multi1)$standard.errors
estadisticos

,(Intercept),sesmiddle,seshigh,write
general,2.445214,-1.2018081,-2.261334,-2.705562
vocation,4.484769,0.6116747,-1.649967,-5.112689


In [43]:
pvalues=(1-pnorm(abs(estadisticos),0,1))*2
pvalues

,(Intercept),sesmiddle,seshigh,write
general,0.0144766100,0.2294379,0.02373856,6.818902e-03
vocation,0.0000072993,0.5407530,0.09894976,3.176045e-07


In [26]:
multi1

Call:
multinom(formula = prog_2 ~ ses + write, data = df)

Coefficients:
         (Intercept)  sesmiddle    seshigh      write
general     2.852198 -0.5332810 -1.1628226 -0.0579287
vocation    5.218260  0.2913859 -0.9826649 -0.1136037

Residual Deviance: 359.9635 
AIC: 375.9635 

In [37]:
ls(multi1)

[1] "AIC"           "call"          "censored"      "coefnames"    
 [5] "conn"          "contrasts"     "convergence"   "decay"        
 [9] "deviance"      "edf"           "entropy"       "fitted.values"
[13] "lab"           "lev"           "n"             "nconn"        
[17] "nsunits"       "nunits"        "rank"          "residuals"    
[21] "softmax"       "terms"         "value"         "vcoefnames"   
[25] "weights"       "wts"           "xlevels"

In [21]:
multi1$coefnames

[1] "(Intercept)" "sesmiddle"   "seshigh"     "write"

In [44]:
ls(summary(multi1))

[1] "AIC"             "call"            "censored"        "coefficients"   
 [5] "coefnames"       "conn"            "contrasts"       "convergence"    
 [9] "decay"           "deviance"        "digits"          "edf"            
[13] "entropy"         "fitted.values"   "is.binomial"     "lab"            
[17] "lev"             "n"               "nconn"           "nsunits"        
[21] "nunits"          "rank"            "residuals"       "softmax"        
[25] "standard.errors" "terms"           "value"           "vcoefnames"     
[29] "weights"         "wts"             "xlevels"

In [25]:
summary(multi1)$Std.Errors

NULL

In [28]:
probabilidades=multi1$fitted.values
head(probabilidades,10)

,academic,general,vocation
1,0.1482764,0.3382454,0.5134781
2,0.1202017,0.1806283,0.6991700
3,0.4186747,0.2368082,0.3445171
4,0.1726885,0.3508384,0.4764731
5,0.1001231,0.1689374,0.7309395
6,0.3533566,0.2377976,0.4088458
7,0.1562562,0.1973504,0.6463934
8,0.1001231,0.1689374,0.7309395
9,0.2331292,0.2203976,0.5464732
10,0.1699402,0.2025531,0.6275067


**TAREA: Estimar las probabilidades del modelo1 usando predict. Segundo, construir la matriz de confusion; después, estimar las métricas que hemos visto Accuracy,VP,FP...**